In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd drive/MyDrive/'...Your path...'/

In [3]:
# used packages
import pandas as pd
import numpy as np
from datetime import datetime, date, time, timedelta
from dateutil.relativedelta import relativedelta
import statistics

In [ ]:
# adobe microsoft salesforce servicenow
# apple ibm nvidia qualcomm

# read the file
name_comp = 'adobe'
path = f'classified_tweets/{name_comp}.csv'
df = pd.read_csv(path, lineterminator='\n')
df.head()

# Form Daily Data

In [ ]:
df = df.loc[:, ('date', 'BERT_sent', 'finsent')]

In [ ]:
# get means for 'BERT_sent' and 'finsent' every day from 01.01.2007 to 31.12.2021

dataDF = pd.DataFrame(columns = ['date', 'BERT_sent', 'finsent'])
dataDF

current_date = date(2007, 1, 1)
delta = timedelta(days=1)
enddate = date(2022, 1, 1)
i = 0
while current_date < enddate:

  day_value = f'{current_date.day}'
  month_value = f'{current_date.month}'

  if len(str(current_date.day)) < 2:
    day_value = f'0{current_date.day}'

  if len(str(current_date.month)) < 2:
    month_value = f'0{current_date.month}'

  day_str = '{0}-{1}-{2}'.format(current_date.year, month_value, day_value )

  dataDF.loc[i, 'date']  = day_str
  temp_df = df[df['date'] == day_str]

  if temp_df.shape[0] > 0:
    dataDF.loc[i, 'BERT_sent']  = statistics.mean(temp_df['BERT_sent'])
    dataDF.loc[i, 'finsent']  = statistics.mean(temp_df['finsent'])
  else:
    dataDF.loc[i, 'BERT_sent']  = 0
    dataDF.loc[i, 'finsent']  = 0

  current_date = current_date+delta
  i += 1

dataDF.head()

In [ ]:
full_data = dataDF.loc[:, ('finsent', 'BERT_sent')]
full_data.head()

In [ ]:
# combine daily sentiment values and financial ones

def daily_fin_data(full_data, value_data, value):
  j = 0
  for i in range(value_data.shape[0]):
    sent_str = dataDF['date'][j].split('-')
    date_sent = date(int(sent_str[0]), int(sent_str[1]), int(sent_str[2]))
    date_value = date(int(value_data['y'][i]), int(value_data['m'][i]), int(value_data['d'][i]))

    while(date_sent <= date_value and j < dataDF.shape[0]-1):
      full_data.loc[j, value] = value_data[value][i]

      if value == 'FLq':
        full_data.loc[j, 'd'] = int(sent_str[2])
        full_data.loc[j, 'm'] = int(sent_str[1])
        full_data.loc[j, 'y'] = int(sent_str[0])
        full_data.loc[j, 'date'] = date_sent

      j+=1
      sent_str = dataDF['date'][j].split('-')
      date_sent = date(int(sent_str[0]), int(sent_str[1]), int(sent_str[2]))

  for k in range(j-1, dataDF.shape[0]):

    sent_str = dataDF['date'][k].split('-')
    date_sent = date(int(sent_str[0]), int(sent_str[1]), int(sent_str[2]))

    full_data.loc[k, value] = value_data[value][i]

    if value == 'FLq':
      full_data.loc[k, 'd'] = int(sent_str[2])
      full_data.loc[k, 'm'] = int(sent_str[1])
      full_data.loc[k, 'y'] = int(sent_str[0])
      full_data.loc[k, 'date'] = date_sent

  return full_data

In [ ]:
months_num = {"m": {"January": 1, "February": 2, "March": 3, "April": 4, "May": 5, "June": 6, "July": 7, "August": 8, "September": 9, "October": 10, "November": 11,  "December": 12}}
list_file = ['flq', 'fly', 'growth', 'pb', 'returns', 'revenue_gr', 'size', 'tatq', 'taty', 'pe', 'vol', 'volume'] # file names
list_value = ['FLq', 'FLy', 'E_GR', 'PB', 'RTN', 'RVN_GR', 'SIZE', 'TATq', 'TATy', 'PE', 'VLT', 'VLM'] # value names

for i in range(len(list_file)):

  name_value = list_file[i]
  value = list_value[i]

  path_value = f'econom_data/{name_comp}/{name_value}.txt'
  value_data = pd.read_csv(path_value, header = 0, delimiter=",")
  value_data = value_data.replace(months_num) # replace name of mouths to numeric
  value_data.drop_duplicates()

  if value_data[value_data.isnull().values == True].shape[0] > 0:

    if value in ['E_GR', 'RVN_GR', 'PE']:
      value_data.fillna(0, inplace=True) # replace NaN values to 0
    else:
      value_data[value].fillna(value_data[value].interpolate(), inplace=True) # replace NaN values to interpolate data

  if value in ['SIZE', 'VLM']: # get correct values for size and volume
    value_data.loc[value_data['n']=='M', value] *= 1000000
    value_data.loc[value_data['n']=='B', value] *= 1000000000
    value_data.loc[value_data['n']=='T', value] *= 1000000000000

  value_data = value_data.sort_index(axis=0, ascending=False)
  value_data = value_data.reset_index(drop=True)

  full_data = daily_fin_data(full_data, value_data, value)

full_data.head()

In [ ]:
# read file with common financial data
path_file = f'econom_data/common_fin_data.csv'
com_fin = pd.read_csv(path_file)
com_fin.head()

In [ ]:
# get final result for a company
result = pd.concat([full_data, com_fin], axis=1)
result.head()

In [ ]:
# save_path = f'panel_data/data_daily/{name_comp}.csv'
# result.to_csv(save_path, header=True, index=False)

# Form Monthly Data

In [ ]:
monDF = pd.DataFrame(columns = result.columns)
monDF

In [ ]:
# calculate monthly data for a company
current_date = date(2007, 1, 1)
delta = relativedelta(months=1)
enddate = date(2022, 1, 1)
i = 0
while current_date < enddate:

  monDF.loc[i, 'd'] = current_date.day
  monDF.loc[i, 'm'] = current_date.month
  monDF.loc[i, 'y'] = current_date.year
  monDF.loc[i, 'y'] = current_date.year
  monDF.loc[i, 'date'] = current_date

  temp_df = result[(result['m'] == current_date.month) & (result['y'] == current_date.year)]

  for col in monDF.columns:
    if col not in ['d', 'm', 'y', 'date']:
      monDF.loc[i, col]  = statistics.mean(temp_df[col])

  current_date = current_date+delta
  i += 1

monDF.head()

,BERT_sent,finsent,FLq,FLy,E_GR,PB,RTN,RVN_GR,SIZE,TATq,...,IR3m,IR6m,IR1y,IS,USIPI,VIX,d,m,y,date
0,0.186022,0.487634,0.0,0.0,45.78,4.50871,-0.442903,22.62,23228064516.129032,0.4462,...,5.106774,5.151935,5.05871,16.198387,99.86,11.148065,1.0,1.0,2007.0,2007-01-01
1,0.217674,0.467463,0.0,0.0,45.78,4.498071,0.190714,22.62,23165000000.0,0.4462,...,5.168214,5.161429,5.057143,15.975,100.81,11.046071,1.0,2.0,2007.0,2007-02-01
2,0.320806,0.686324,0.0,0.0,45.78,4.65429,0.489355,22.62,23775483870.967743,0.46,...,5.078065,5.099355,4.922903,0.864194,101.03,15.18871,1.0,3.0,2007.0,2007-03-01
3,0.303499,0.66246,0.0,0.0,45.78,4.8773,-0.069667,22.62,24914333333.333332,0.46,...,5.000667,5.068333,4.932667,0.32,101.75,13.036333,1.0,4.0,2007.0,2007-04-01
4,0.357868,0.674533,0.0,0.0,45.78,4.859935,0.270645,22.62,24839032258.064518,0.46,...,4.871935,4.983548,4.914839,-4.945806,101.81,13.353548,1.0,5.0,2007.0,2007-05-01


In [ ]:
# save_path = f'panel_data/data_monthly/{name_comp}.csv'
# monDF.to_csv(save_path, header=True, index=False)

# Form Panel Data

In [68]:
# form monthly panel data for an industry

def get_panel_data(company, companies_list, save_path, number_value):

  values = ['finsent', 'BERT_sent', 'date', 'FLq', 'FLy', 'E_GR', 'PB', 'RTN', 'RVN_GR', 'SIZE', 'TATq', 'TATy',
            'VLT', 'VLM', 'BCI', 'CCIm', 'CCIy', 'IR1m', 'IR3m', 'IR6m', 'IR1y', 'IS', 'USIPI', 'VIX', 'PE']

  time_temp = []
  for i in range(number_value):
    time_temp.append(i)
  time = np.array(time_temp*number_value)

  panel_df = pd.DataFrame(zip(company, time), columns=['company', 'time'])
  panel_df.set_index(['company', 'time'], inplace=True)

  for name_comp in companies_list:
    path = f'panel_data/data_monthly/{name_comp}.csv'
    mon_df = pd.read_csv(path)
    mon_df['SIZE']= np.log(mon_df['SIZE'])
    mon_df['VLM']= np.log(mon_df['VLM'])
    for name_value in values:
      panel_df.loc[name_comp, name_value] = list(mon_df[name_value])

  panel_df.to_csv(save_path, header=True, index=True)

In [69]:
# software industry
number_value = 180
company = np.array(['adobe']*number_value+['microsoft']*number_value+['salesforce']*number_value+['servicenow']*number_value)
companies_list = ['adobe', 'microsoft', 'salesforce', 'servicenow']
save_path = f'panel_data/software.csv'

get_panel_data(company, companies_list, save_path, number_value)

In [71]:
# telecom industry
number_value = 180
company = np.array(['apple']*number_value+['ibm']*number_value+['nvidia']*number_value+['qualcomm']*number_value)
companies_list = ['apple', 'ibm', 'nvidia', 'qualcomm']
save_path = f'panel_data/telecom.csv'

get_panel_data(company, companies_list, save_path, number_value)